In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
from qiskit.circuit.quantumcircuit import QubitSpecifier
from qiskit.circuit import Gate
from qiskit.visualization import plot_histogram
from typing import List

import numpy as np
import random

from ryQFTAdder import *

# Find the Largest Number (basic ver.)

This part demonstrate the quantum algorithm to solve task 1

## QRAM

We begin with defining how the data in the list can be accessed quantumly. Though a QRAM is not required for solving the original task 1, it will be used nontrivially when we extent the problem in the next section.

We want a circuit $U_f$ such that:

$$
\ket{i}\ket{0} \xrightarrow{U_f} \ket{i}\ket{f(i)}
$$

where $i$ is the index and $f(i)$ gives the ith number in the list.

For each integer in the list, we choose basic/state encoding, the highest bit is sign bit. For example, 5 and -6 can be encoded as:

$$\begin{align*}
5  &\rightarrow \ket{00101} \\
-6 &\rightarrow \ket{11010}      
\end{align*}$$

4 bits is enough to hold 5 and -6. Since we'll later do addition and subtraction between two numbers in the list, we use 1 extra bit to prevent overflow. The indeices will be encoded by state encoding and we encode the them into the ctrl bits of mcx gate. The control bit will be controlled by either '0' or '1' based on the state encoding of the index on that position.

In [2]:
"""
Here is a general method to convert a list of number into quantum data.

In this function, we used np.max to find the bound for the list, 
but such information is not encoded into the QRAM so the algorithm that we'll 
present in a moment to solve the problem will not lost any generality. And in practice 
the bound on integer is always predetermined by its type like 'int' in C usually represents 
integers from -2147483648 to 2147483647. The reason that we choose to dynamically determine 
the number of qubit is for compactness and clarity.

"""

def toQRAM(
    data_list: List[int],
    num_idx_cbit: int = None,
    num_data_cbit: int = None
) -> QuantumCircuit:
    """
        returns the circuit U_f such that U_f|i>|0> = |i>|x_i>

        Args:
            data_list:
                the list of number needs to be encoded

            num_data_cbit:
                size of the classical register of for the data qubit
    """
    
    # first construct qram as a gate 

    # define minimum number of qubit to represent all the integers in the list
    bound = np.max(np.abs(data_list))
    num_qubit_i = int(np.ceil(np.log2(len(data_list))))
    num_qubit_xi = len(bin(bound)[2:]) + 2 # one for sign and one for preventing overflow

    if None == num_idx_cbit:
        num_idx_cbit = num_qubit_i
    if None == num_data_cbit:
        num_data_cbit = num_qubit_xi
    
    # define index and data quantum registers
    reg_i  = QuantumRegister(num_qubit_i)
    reg_xi = QuantumRegister(num_qubit_xi)
    
    qc = QuantumCircuit(reg_i, reg_xi, name="QRAM")

    for i, x in enumerate(data_list):
        # convert index into binary and agree with the number of index qubit
        i_bin = format(i, f'0{num_qubit_i}b') 
        # convert x_i into binary and agree with the number of data qubit
        x_bin = bin(x & int('1' * num_qubit_xi, 2))[2:]

        # determine which control bit is controlled by '0'
        for i_bin_idx, i_bin_symbol in enumerate(reversed(i_bin)):
            if '0' == i_bin_symbol:
                qc.x(reg_i[i_bin_idx])
        
        # encode x_i
        for x_bin_idx, x_bin_symbol in enumerate(reversed(x_bin)):
            if '1' == x_bin_symbol:
                qc.mcx(reg_i, reg_xi[x_bin_idx])
        
        # uncompute for the control bit
        for i_bin_idx, i_bin_symbol in enumerate(reversed(i_bin)):
            if '0' == i_bin_symbol:
                qc.x(reg_i[i_bin_idx])

    # define qram as a gate
    qram_gate = qc.to_gate()

    # using the qram gate, construct the full circuit 
    q_i  = QuantumRegister(num_qubit_i, "q_i")
    q_xi = QuantumRegister(num_qubit_xi, "q_xi")
    c_i = ClassicalRegister(num_idx_cbit, "i")
    c_xi = ClassicalRegister(num_data_cbit, "xi") 

    full_circ = QuantumCircuit(q_i, q_xi, c_i, c_xi)
    full_circ.append(qram_gate, range(full_circ.num_qubits))

    return full_circ

For example the list [-3,2,5,-1] in the qram form will be:

In [3]:
ex_list = [-3, 2, 5, 1]

qram_circ = toQRAM(ex_list)
qram_circ.decompose().draw()

┌───┐                    ┌───┐          ┌───┐          ┌───┐     
 q_i_0: ┤ X ├──■────■────■────■──┤ X ├───────■──┤ X ├──■────■──┤ X ├──■──
        ├───┤  │    │    │    │  ├───┤┌───┐  │  ├───┤  │    │  └───┘  │  
 q_i_1: ┤ X ├──■────■────■────■──┤ X ├┤ X ├──■──┤ X ├──■────■─────────■──
        └───┘┌─┴─┐  │    │    │  └───┘└───┘  │  └───┘┌─┴─┐  │       ┌─┴─┐
q_xi_0: ─────┤ X ├──┼────┼────┼──────────────┼───────┤ X ├──┼───────┤ X ├
             └───┘  │    │    │            ┌─┴─┐     └───┘  │       └───┘
q_xi_1: ────────────┼────┼────┼────────────┤ X ├────────────┼────────────
                  ┌─┴─┐  │    │            └───┘          ┌─┴─┐          
q_xi_2: ──────────┤ X ├──┼────┼───────────────────────────┤ X ├──────────
                  └───┘┌─┴─┐  │                           └───┘          
q_xi_3: ───────────────┤ X ├──┼──────────────────────────────────────────
                       └───┘┌─┴─┐                                        
q_xi_4: ────────────────────┤ X ├────────────────────────────────────────
                            └───┘                                        
   i: 2/═════════════════════════════════════════════════════════════════
                                                                         
  xi: 5/═════════════════════════════════════════════════════════════════

And we can confirm the qram indeed stores the desired data by making queries on all possible indices:

In [4]:
query_circ = QuantumCircuit(qram_circ.qregs[0])
# equal superposition on all indices
query_circ.h(query_circ.qregs[0])

# add the query in fron of the qram
query_circ = qram_circ.compose(query_circ, qram_circ.qregs[0], front=True)
# query_circ.append(qAdder(-1, query_circ.qregs[1].size), query_circ.qregs[1])

# measure and get results of the queries
query_circ.measure(query_circ.qregs[0], query_circ.cregs[0])
query_circ.measure(query_circ.qregs[1], query_circ.cregs[1])

query_circ.draw()

┌───┐┌───────┐┌─┐                  
 q_i_0: ┤ H ├┤0      ├┤M├──────────────────
        ├───┤│       │└╥┘┌─┐               
 q_i_1: ┤ H ├┤1      ├─╫─┤M├───────────────
        └───┘│       │ ║ └╥┘┌─┐            
q_xi_0: ─────┤2      ├─╫──╫─┤M├────────────
             │       │ ║  ║ └╥┘┌─┐         
q_xi_1: ─────┤3 QRAM ├─╫──╫──╫─┤M├─────────
             │       │ ║  ║  ║ └╥┘┌─┐      
q_xi_2: ─────┤4      ├─╫──╫──╫──╫─┤M├──────
             │       │ ║  ║  ║  ║ └╥┘┌─┐   
q_xi_3: ─────┤5      ├─╫──╫──╫──╫──╫─┤M├───
             │       │ ║  ║  ║  ║  ║ └╥┘┌─┐
q_xi_4: ─────┤6      ├─╫──╫──╫──╫──╫──╫─┤M├
             └───────┘ ║  ║  ║  ║  ║  ║ └╥┘
   i: 2/═══════════════╩══╩══╬══╬══╬══╬══╬═
                       0  1  ║  ║  ║  ║  ║ 
  xi: 5/═════════════════════╩══╩══╩══╩══╩═
                             0  1  2  3  4

In [5]:
print("Original list: ", ex_list)

# run the circuit to get the result of the queries
simulator = Aer.get_backend('qasm_simulator')
result = execute(query_circ, simulator).result()
counts = result.get_counts()
# raw output in binary
print("Query raw data: ", counts)

# extract original classical data 
print("Decoded result: ")
for xi_i in counts:
    xi_bin, i_bin = xi_i.split(" ")
    
    # have to consider the sign when convert back into integer 
    sign_bit = xi_bin[0]
    xi = int(xi_bin[1:], 2)
    if '1' == sign_bit:
        xi = -(2**(len(xi_bin)-1) - xi)
    # index is unsigned
    i = int(i_bin, 2)

    print(f"f[{i}]: {xi}")

Original list:  [-3, 2, 5, 1]
Query raw data:  {'11101 00': 263, '00010 01': 249, '00001 11': 252, '00101 10': 260}
Decoded result: 
f[0]: -3
f[1]: 2
f[3]: 1
f[2]: 5


We can see the recovered data is exactly the same as the original list

## Quantum-classical Adder

With classically given integer X, positive or negative, we want the adder to do:

$$
    \ket{n} \xrightarrow{} \ket{n + X}
$$

There are many different methods, here we choose an implementation in fourier basis. Both the fourier transform defined by y-rotation and the addition circuit in such basis are proposed by Rich Rines & Isaac Chuang^[1]. Detailed implementation is in `ryQFTAdder.py`.

[1]: Rines, R., & Chuang, I. (2018). High Performance Quantum Modular Multipliers. [arXiv:1801.01081](https://arxiv.org/abs/1801.01081).

## Quantum Comparator

Now we want to implement a gate $U_{a<X}$ that encode the comparison result into phase:
- partial version
$$
U_{a<X,p}\ket{a} = (-1)^{a<X}\ket{a-X}
$$ 

- complete version
$$
U_{a<X,c}\ket{a} = (-1)^{a<X}\ket{a}
$$

They only differ by a quantum addition of `X`. The key idea for building both is to apply a `Z` gate on the highest data qubit after adding `-X` to $\ket{a}$. When the highest bit of `(a - X)` is $1$, $a-X<0$, $a<X$, $-1$ phase is added. When the highest bit of `(a - X)` is $0$, $Z\ket{0}=\ket{0}$, no phase is added.

### controlled quantum comparator

The controlled quantum comparator is a unitary, $\text{c-}U_{a<X}$, that applys quantum comparator when the control bit is $1$:

- partial version
$$
\text{c-}U_{a<X,p}\ket{i}\ket{a} = (-1)^{(a<X)\cdot i}\ket{i}\ket{a-X}
$$

- full version
$$
\text{c-}U_{a<X,c}\ket{i}\ket{a} = (-1)^{(a<X)\cdot i}\ket{i}\ket{a}
$$

to get a controlled comparator, we only have to change the `Z` gate in the quantum comparator to a `CZ` gate controlled by the control bit $\ket i$.

In [6]:
def partial_ctl_comparator(
    X: int,
    circuit: QuantumCircuit,
    ctl_qubit: QubitSpecifier,
    target_reg: QuantumRegister
):
    """
        |i>|a>  ---> (-1)^{(a<X)*i} |a - X >

    """

    # |a> to | a - X >
    circuit.append(qAdder(-X, target_reg.size), target_reg)
    circuit.cz(ctl_qubit, target_reg[-1])

    return

def complete_ctl_comparator(
    X: int,
    circuit: QuantumCircuit,
    ctl_qubit: QubitSpecifier,
    target_reg: QuantumRegister
):
    """
        |i>|a>  ---> (-1)^{(a<X)*i} |a>

    """

    # |a> to | a - X >
    circuit.append(qAdder(-X, target_reg.size), target_reg)
    circuit.cz(ctl_qubit, target_reg[-1])
    circuit.append(qAdder(X, target_reg.size), target_reg)

    return

Here is an example of a controlled partial comparator circuit. The result of `(-5 < -2)` will be stored in control qubit's phase when the control qubit is $\ket{1}$.

In [7]:
test_comp_circ = toQRAM([-5, 1], 0, 0)
test_comp_circ.barrier()
test_comp_circ.add_register(QuantumRegister(1, name="ctl"))
partial_ctl_comparator(
    X = -2,
    circuit= test_comp_circ,
    ctl_qubit= test_comp_circ.qregs[-1][0],
    target_reg= test_comp_circ.qregs[1]
)

test_comp_circ.draw()

┌───────┐ ░               
   q_i: ┤0      ├─░───────────────
        │       │ ░ ┌─────────┐   
q_xi_0: ┤1      ├─░─┤0        ├───
        │       │ ░ │         │   
q_xi_1: ┤2      ├─░─┤1        ├───
        │  QRAM │ ░ │         │   
q_xi_2: ┤3      ├─░─┤2 add(2) ├───
        │       │ ░ │         │   
q_xi_3: ┤4      ├─░─┤3        ├───
        │       │ ░ │         │   
q_xi_4: ┤5      ├─░─┤4        ├─■─
        └───────┘ ░ └─────────┘ │ 
   ctl: ────────────────────────■─

## Solution

The general idea is to implement a variation of the Deutsch's Algorithm. We first put $\ket 0$ to superposition: $H\ket{0}\rightarrow {1\over\sqrt{2}}(\ket{0}+\ket{1})$. Then we can use controlled quantum comparator and another hadamard gate to exract the value of `(a < b)`.

### algorithm

Now we have all the ingredients. Here's the algorithm to find the largest number from a list two integers `[a, b]` :

1. Let the index qubit in the default $\ket 0$ state. Apply qram circuit for the list `[a, b]`.
2. Use an extra qubit in $\ket{+}$ state to apply a (partial) controlled quantum comparator with `X = b` on the data register of the qram.
3. Apply `H` on the extra qubit.
4. Measure the extra qubit.
5. If the measurement result is `1`, `b` is the largest. If the measurement result is `0`, `a` is the largest.

### explanation

$$ 
\ket{0}\ket{a} 
\xrightarrow{H} {1\over\sqrt{2}}(\ket{0} + \ket{1})\ket{a} 
\xrightarrow{cU_{a<b,p}} {1\over\sqrt{2}}(\ket{0} +(-1)^{a<b}\ket{1})\ket{a-b}
\xrightarrow{H} \ket{a<b}\ket{a-b}
$$

### analysis

- The success rate for this algorithm is $1$ and we only have to run the quantum circuit for once.
- The number of qubit required for the algorithm is $O(\log(\max(a,b)))$ due to the implementation of qram.
- This algorithm works for all kinds of number. Because the qram, quantum-classical adder and the quantum comparator work for all integers.

### implementation


In [8]:
def find_the_largest_number(a, b, show_process: bool = False) -> int:

    num_list = [a,b]

    # step 1. |0>|a>
    qc = toQRAM(num_list, 0, 0)

    # step 2. |0>|a>(|0> + |1>)/√2  --> |0>|(a - b)>(|0> + (-1)^(a<b)|1>)/√2
    qc.add_register(QuantumRegister(1, name= "comp"))
    qc.h(qc.qregs[-1])
    qc.add_register(ClassicalRegister(1, name= "a<b"))

    partial_ctl_comparator(
        X          = b,
        circuit    = qc,
        ctl_qubit  = qc.qregs[-1][0],
        target_reg = qc.qregs[1]
    )

    # step 3. |0>|(a - b)>(|0> + (-1)^(a<b) |1>)/√2 --> |0>|(a - b)>|(a<b)>
    qc.h(qc.qregs[-1])

    # step 4. get measurement result of (b>a)
    qc.measure(qc.qregs[-1], qc.cregs[-1])
    # we only need to run the circuit once
    backend = Aer.get_backend('qasm_simulator')
    comp_result = execute(qc, backend,shots=1).result().get_counts()
    aLb = comp_result.most_frequent()[0]

    # optional print
    if show_process:
        print(qc)
        print("raw data: ", comp_result)
        print(f"a<b: {aLb}")

    # step 5. decide which one is the largest
    if '1' == aLb:
        return b

    return a

### some test cases

In [9]:
find_the_largest_number(5, -6, show_process=True)

        ┌───────┐                      
   q_i: ┤0      ├──────────────────────
        │       │┌─────────┐           
q_xi_0: ┤1      ├┤0        ├───────────
        │       ││         │           
q_xi_1: ┤2      ├┤1        ├───────────
        │  QRAM ││         │           
q_xi_2: ┤3      ├┤2 add(6) ├───────────
        │       ││         │           
q_xi_3: ┤4      ├┤3        ├───────────
        │       ││         │           
q_xi_4: ┤5      ├┤4        ├─■─────────
        └─┬───┬─┘└─────────┘ │ ┌───┐┌─┐
  comp: ──┤ H ├──────────────■─┤ H ├┤M├
          └───┘                └───┘└╥┘
 a<b: 1/═════════════════════════════╩═
                                     0 
raw data:  {'0  ': 1}
a<b: 0


5

In [10]:
def test_find_the_largest_number(
    int_range: int,
    num_tests: int
):  
    """
        For two randomly generated integers in range `[-int_range, int_range]`, run the quantum algorithm vs the classical max function
        to confirm the result is correct. The test will be run for `num_tests` times.
    """
    print("Case a = b:")
    for i in range(num_tests):
        a = random.randint(-int_range, int_range)
        b = a

        qMax = find_the_largest_number(a,b)
        cMax = max(a,b)
        print(f"testcase-{i} [{a}, {b}]: qMax = {qMax}, cMax = {cMax}. Results agree: {qMax == cMax}")


    # random tests
    print("\nRandom test (a>b or a<b):")
    for i in range(num_tests):
        a = random.randint(-int_range, int_range)
        b = random.randint(-int_range, int_range)

        qMax = find_the_largest_number(a,b)
        cMax = max(a,b)
        print(f"testcase-{i} [{a}, {b}]: qMax = {qMax}, cMax = {cMax}. Results agree: {qMax == cMax}")

    return 

In [11]:
test_find_the_largest_number(128, 5)

Case a = b:
testcase-0 [-34, -34]: qMax = -34, cMax = -34. Results agree: True
testcase-1 [93, 93]: qMax = 93, cMax = 93. Results agree: True
testcase-2 [-82, -82]: qMax = -82, cMax = -82. Results agree: True
testcase-3 [-13, -13]: qMax = -13, cMax = -13. Results agree: True
testcase-4 [-31, -31]: qMax = -31, cMax = -31. Results agree: True

Random test (a>b or a<b):
testcase-0 [59, -32]: qMax = 59, cMax = 59. Results agree: True
testcase-1 [-128, -127]: qMax = -127, cMax = -127. Results agree: True
testcase-2 [2, -65]: qMax = 2, cMax = 2. Results agree: True
testcase-3 [11, 121]: qMax = 121, cMax = 121. Results agree: True
testcase-4 [21, 125]: qMax = 125, cMax = 125. Results agree: True


## Conclusion

Analysis and data both show that the proposed algorithm do solve the problem.

# Find the Largest Number (extented ver.)